# Конфигурация Kafka Producer

## Подготовка

Создать топик

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-producer-demo-topic \
    --create \
    --partitions 2 \
    --replication-factor 3

Проверить, что топик доступен:

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic my-producer-demo-topic \
    --describe

In [ ]:
zookeeper-shell $ZOOKEEPER_HOST:$ZOOKEEPER_PORT \
    get /brokers/topics/my-producer-demo-topic |
  tail -n 1 | json_pp

In [ ]:
curl -Ls http://redpanda:8080/api/topics/my-producer-demo-topic/partitions | json_pp

## Настройка продюсера

### Общая информация

Основная конфигурация продюсера включает в себя использование следующих настроек (*):

| Имя | Описание | Значение по умолчанию | Допустимые значения |
|-----|----------|-------------|----|
| [`client.id`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#client-id) | метка продюсера, чтобы можно было отследить из какого источника пришли данные | пустая строка | строки |
| [`retries`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#retries) | Сколько раз повторять отправку батча в Kafka, прежде чем сообщить клиенту о неудаче | MAX_INT | Целые числа >= 0 |
| [`acks`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#acks) | Сколько реплик должно подтвердить запись сообщения, прежде чем брокер ответит клиенту, что сообщения записались | all (-1) | all, 1, 0, -1 |
| [`batch.size`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#batch.size) | Максимальный размер батча в байтах, который можно отправить в одном запросе на запись к брокеру | 16384 | Целые числа >= 0 |
| [`linger.ms`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#linger.ms) | Сколько времени можно тратить на формирование одного батча на стороне клиента | 0 | Целые числа >= 0 |
| [`enable.idempotence`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#enable-idempotence) | Если установлено в true, то брокер убедится, что только одна копия данных будет записана не смотря на возможные сбои | true | true, false |
| [`max.in.flight.requests.per.connection`](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html#max-in-flight-requests-per-connection) | Сколько батчей продюсер может отправить без не дожидаясь подтверждения доставки. Если `max.in.flight.requests.per.connection` > 0 и `retries` > 0 и `enable.idempotence == false`, то возможно нарушение порядка сообщений | 5 ||

(*) Настроек [на много больше](https://docs.confluent.io/platform/current/installation/configuration/producer-configs.html), но на практике наиболее часто используют только перечисленные

### Функция для работы с продюсером

Функция `kafka_producer` определена для исключения дублирования кода:

In [ ]:
function kafka_producer() {
    local config_file="${1:-/tmp/producer.cfg}"
    
    local HOST=kafka1

    execute "[ -f ${config_file} ]" || {
        echo "[WARN] No '${config_file}' file found. An empty one will be created" >&2
        execute touch "${config_file}"
    }
    
    kafka-console-producer \
      --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
      --topic my-producer-demo-topic \
      --property "parse.key=true" \
      --property "key.separator=:" \
      --producer.config ${config_file} \
<<EOF
0:The Shawshank Redemption, 1994
0:God Father, 1972
0:The Dark Knight, 2008
0:The Godfather Part II, 1974
0:12 Angry Men, 1957
1:Schindler's List, 1993
1:The Lord of the Rings: The Return of the King, 2003
1:Pulp Fiction, 1994
1:The Lord of the Rings: The Fellowship of the Ring, 2001
1:The Good, the Bad and the Ugly, 1966
EOF
}

Функиця `check_offset` позволит посмотреть оффсеты в партициях топика:

In [ ]:
function check_offset() {
    local topic=${1:-my-producer-demo-topic}

    kafka-run-class kafka.tools.GetOffsetShell \
        --broker-list $KAFKA_HOST:$KAFKA_PORT \
        --topic ${topic} \
        --time -1
}

### Запуск продюсера с настройками по умолчанию

Для `kafka-console-producer` настройками по умолчанию являются:
- `retries == MAX_INT`
- `acks == all`

Так можно заключить, что при использовании настроек по умолчанию клиент будет повторять запись одного и того же батча до тех пор, пока все реплики не ответят, что данные были записаны

In [ ]:
check_offset

In [ ]:
kafka_producer

In [ ]:
check_offset

### At-Most-Once продюссер

In [ ]:
cat <<EOF | HOST=kafka1 new_file /tmp/producer.at-least-once.cfg
acks=1
retries=0
EOF

In [ ]:
HOST=kafka1 execute \
cat /tmp/producer.at-least-once.cfg

In [ ]:
check_offset

In [ ]:
kafka_producer /tmp/producer.at-least-once.cfg

In [ ]:
check_offset

### Задание

1. Отправить сообщения в Kafka в `At-Least-Once` семантике
1. Придумать новую конфигурацию для `At-Most-Once` семантики

## Работа продюссера при сбоях кластера

Архитектура Apache Kafka реализована по шаблону Leader-Follower: все запросы на запись и чтение выполняются через Leader реплику. Follower реплики поддерживают полную копию данных лидера. В случае, если сервер, на котором находится Leader реплика выходит из строя, то Apache Kafka автоматически находит нового лидера и обеспечивает требуемое количество Follower реплик на других серверах.

### Настройка min.insync.replicas

При создании топика `my-producer-demo-topic` явно были указаны следующие параметры:

- количество реплик (сколько раз дублировать данные в кластере): 3
- количество партиций: 2

Дополнительно, топик наследует конфигурацию Kafka кластера по умолчанию. Полный список конфигов можно посмотреть через команду `kafka-configs` или через [web интерфейс](http://localhost:8080/topics/my-producer-demo-topic#configuration). На данный момент интересует лишь один конфиг: `min.insync.replicas`.

Настройка `min.insync.replicas` позволяет дополнительно со стороны Kafka кластера указать сколько реплик должны ответить "OK" на каждый запрос от продюссера с `acks=ALL`. Это позволяет еще лучше защитить данные в кластере на случай сбоя. Настройку `min.insync.replicas` можно указывать как на уровне всего кластера, и все топики будут наследовать ее, либо можно настроить ее на уровне топика.

Команда `kafka-configs` позволяет посмотреть значения настроек различных сущностей кластера, в том числе и топиков:

In [ ]:
kafka-configs --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --describe \
    --all \
    --entity-name my-producer-demo-topic \
    --entity-type topics |
  grep min.insync.replicas

При создании топика `my-producer-demo-topic` нигде явно не было указано, что его конфиг `min.insync.replicas` должен быть равен единице, а значит он унаследовал значение `min.insync.replicas=1` всего кластера.
Это значение можно изменить на уровне топика:

In [ ]:
kafka-configs --bootstrap-server "${KAFKA_HOST}":"${KAFKA_PORT}" \
    --alter \
    --entity-type topics \
    --entity-name my-producer-demo-topic \
    --add-config min.insync.replicas=3

Новое значение `min.insync.replicas` топика `my-producer-demo-topic` равно `3`, значит при записи брокер должен записать данные во все 3 реплики при запросе `acks=ALL`:

In [ ]:
kafka-configs --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --describe \
    --all \
    --entity-name my-producer-demo-topic \
    --entity-type topics |
  grep min.insync.replicas

In [ ]:
HOST=kafka1 new_file \
/tmp/producer.acks-3.cfg <<EOF
acks=all
EOF

In [ ]:
HOST=kafka1 execute \
cat /tmp/producer.acks-3.cfg

In [ ]:
check_offset
kafka_producer /tmp/producer.acks-3.cfg
check_offset

На данный момент кластер состоит из трех брокеров (по одному docker сервису на каждого) и одного zookeeper. Симуляцией сбоя будет временная остановка одного из docker контейнеров, для этого необходимо выполнить `docker compose pause kafka3`:

In [ ]:
docker compose pause kafka3
docker compose ps kafka3

Дополнительная проверка, что broker недоступен:

In [ ]:
zookeeper-shell $ZOOKEEPER_HOST:$ZOOKEEPER_PORT ls /brokers/ids |
    grep -q '\[1, 2\]' || {
        echo 'Пожалуйста, выполните `docker compose pause kafka3` в терминале'
        false
    }

In [ ]:
check_offset
kafka_producer /tmp/producer.acks-3.cfg
check_offset

Можно заметить, что запись не выполнилась по причине того, что один из брокеров был недоступен, а поэтому количество `acks=all` было невозможно обеспечить

Одним из решений проблемы может быть установка `acks=1` на продюсере:

In [ ]:
HOST=kafka1

new_file /tmp/producer.acks-1.cfg <<EOF
acks=1
EOF

execute \
cat /tmp/producer.acks-1.cfg

In [ ]:
check_offset
kafka_producer /tmp/producer.acks-1.cfg
check_offset

Запись выполнилась успешно, т.к. клиент указал, что ему необходимо получить подтверждение только от лидера (`acks=1`).

Но управлять конфигурацией продюсера не всегда возможно, а поэтому в качестве второго решния решения, можно снизить количество `min.insync.replicas` до двух, и тогда запись выполнится успешно:

In [ ]:
kafka-configs --bootstrap-server "${KAFKA_HOST}":"${KAFKA_PORT}" \
    --alter \
    --entity-name my-producer-demo-topic \
    --entity-type topics \
    --add-config min.insync.replicas=2

In [ ]:
HOST=kafka1 execute \
cat /tmp/producer.acks-3.cfg

check_offset
kafka_producer /tmp/producer.acks-3.cfg
check_offset

Установка значения `min.insync.replicas` на единицу меньше (`--replication-factor - 1`), чем количество реплик топика является хорошей практикой, которая была получена опытным путем. Это не более чем общая рекомендация, на практике каждая ситуация является уникальной, поэтому необходимо осознанно подходить к организации хранения данных

На данный момент брокер `kafka3` все еще недоступен, поэтому если попробовать выполнить запись с опцией `acks=all`, где `all` - это значение `--replication-factor` топика, то запись все равно будет успешной: брокер ответит клиенту, что запись успешна, т.к. он смог получить подтверждение от `min.insync.replicas` реплик, что данные записаны. В этом случае Kafka кластер полагается на то, что администратор знает об этой особенности кластера, и поэтому настроил топик таким образом:

In [ ]:
HOST=kafka1 execute \
cat /tmp/producer.acks-3.cfg

check_offset
kafka_producer /tmp/producer.acks-3.cfg
check_offset

Прежде чем продолжить необходимо восстановить `kafka3`:

In [ ]:
docker compose unpause kafka3

Дополнительная проверка, что брокер вернулся:

In [ ]:
zookeeper-shell $ZOOKEEPER_HOST:$ZOOKEEPER_PORT ls /brokers/ids |
    grep -q '\[1, 2, 3\]' || {
        echo 'Пожалуйста, выполните `docker compose unpause kafka3` в терминале'
        false
    }

## Настройка производительности

Продюсер отправлят данные в Kafka батчами, Так оптимизация продюсера обычно включает в себя настройку двух параметров `linger.ms`, `batch.size`.

Батч отправляется в Kafka при наступлении одного из двух условий:

- Время `linger.ms` на формирование батча закончилось
- Батч превышает размер `batch.size`

Так, уменьшая значение `linger.ms`, остается меньше времени на наполнение батча, а значит больше батчей будет отправлено в Kafka. C другой стороны увеличение значение `linger.ms` ведет к увеличению времени на обработку одной записи.

### Функция для замеров производительности

### Тестирование производительности

Утилита `kafka-producer-perf-test` позволяет протестировать производительность продюсера.

Базовая конфигурация включает в себя следующие настройки и их значения по умолчанию:

| Имя | Параметр | Значение |
|---------------|---|-------------|
| Количество сообщений | `--num-records` | 3000 |
| Размер одного сообщения | `--record-size` | 1000 байт (1КБ) |
| Скорость | `--throughput` | 200 сообщений в секунду |
| Время на формирование батча | `linger.ms` | 0 мс (отправлять сразу) |
| Размер батча | `batch.size` | 16384 байт (16КБ) |

In [ ]:
function producer_perf_test() {
  local linger=${1:-0}
  local batch_size=${2:-16384}

  kafka-producer-perf-test \
    --throughput 200 \
    --record-size 1000 \
    --num-records 3000 \
    --topic perf-test-topic \
    --producer-props linger.ms=${linger} batch.size=${batch_size} bootstrap.servers=$KAFKA_HOST:$KAFKA_PORT \
    --print-metrics | grep \
"3000 records sent\|\
producer-metrics:outgoing-byte-rate\|\
producer-metrics:bufferpool-wait-ratio\|\
producer-metrics:record-queue-time-avg\|\
producer-metrics:request-latency-avg\|\
producer-metrics:batch-size-avg"
}

### Точка осчёта (baseline)

In [ ]:
producer_perf_test

#### Выводы

В примере выше можно наблюдать:
- Cредний размер батча `1163.331` байт (1КБ). Значение в 16 раз ниже, чем ожидаемый размер батча (16КБ). Размер одного сообщения 1КБ, а время ожидания на формирование батча (`linger.ms`) равно 0, то можно отметить, что клиент отправлял сообщения в kafka как только они появлялись (каждый батч содержал только одно сообщение);
- Значение `bufferpool-wait-ratio` равно 0, что сигнализирует о том, что отправка батчей не ждет ответа от предыдущих запросов, следовательно, Kafka брокеры способны обрабатывать запросы на столько быстро, на сколько быстро клиент отправляет их (!);
- Значения `outgoing-byte-rate` и `request-latency-avg` показывают пропускную способность (throughput) брокера и время на обработку одного запроса (latency) брокером
- Значение `record-queue-time-avg` показывает сколько времени сообщения находились в батче, прежде чем были отправлены в Kafka

### Увеличение `linger.ms`

Увеличение `linger.ms` устанавливает время на формирование батча, следовательно, один батч будет содержать больше одного сообщения, а значит будет меньше обращения к брокеру по сети:

In [ ]:
producer_perf_test 100

#### Выводы

После увеличения значения `linger.ms` первое что можно заметить - это увеличение размера батча, которое тем не менее все еще ниже указанного значения `batch.size`.

Следовательно, батч не успевал заполниться до конца, а время `linger.ms` заканчивалось, что вынуждало клиента отправлять неполный батч в брокер.

**Важно**: можно так же заметить, что throughput (`outgoing-byte-rate`) снизился, а latency увеличился (`request-latency-avg`).

**Таким образом,** указанная конфигурация хуже, чем `baseline`

### Увеличение `linger.ms` и `batch.size`

In [ ]:
producer_perf_test 100 300000

#### Выводы

- Средний размер батча увеличился на ~25%
- Средний размер батча все еще ниже указанного значения, значит батчи все еще отправляются неполными
- Пропускная способность снижается, а время на обработку одной записи увеличивается

**Таким образом**, настройки все еще хуже baseline

### Увеличение времени формирования батча `linger.ms`

В двух предыдущих попытках оптимизации было видно, что батчи отправляются неполными, что ведет к избыточному числу обращений к брокеру. Очевидным решением полностью заполнить батч - это увеличение времени на формирование батча (`linger.ms`):

In [ ]:
producer_perf_test 1500 300000

#### Выводы

- средний размер батча близок к ожидаемому значению
- пропускная способность (throughput) ниже baseline
- время на обработку одной записи (latency) выше baseline

### Вывод

| Метрика | linger.ms=0; batch.size=16384 | linger.ms=100; batch.size=16384 | linger.ms=100; batch.size=300000 | linger.ms=1500; batch.size=300000 |
|-----|------|------|------|------|
| Средний размер батча (batch-size-avg) | 1215.030 | 16164.670 | 23720.164 | 275700.182 |
| Пропускная способность (outgoing-byte-rate) | 75594.490 | 68866.596 | 68720.642 | 68246.401 |
| Среднее время сообщения в батче (record-queue-time-avg) | 4.229 | 95.457 | 109.070 | 1406.273 |
| Время на обработку одного сообщения (request-latency-avg) | 43.292 | 53.649 | 63.836 | 226.091 |

Таким образом, **значения конфигов по умолчанию** для приложения, которое отправляет 200 сообщений в секунду, при размере одного сообщения 1000 байт, являются наиболее оптимальными

### Задание

1. Что произойдет, если установить `--throughput -1`?
1. Что произойдет, если увеличить количество сообщений?
1. Получить наиболее оптимальные значения `linger.ms`, `batch.size` для функционирования системы в следующих условиях

Входные данные:
| Имя | Значение |
|-----|------|
| Размер сообщения | 512 байт |
| Пропускная способность | 100 сообщений в секунду |